<a href="https://colab.research.google.com/github/Muhozgu/etl-and-analytics-with-spark-and-powerbi/blob/main/eda_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
63 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-4.1.1/spark-4.1.1-bin-hadoop3.tgz

In [ ]:
!tar xf spark-4.1.1-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [ ]:
!pip install py4j

In [ ]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.1.1-bin-hadoop3"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

In [ ]:
df = spark.read.csv(
    "/content/bmw_data.csv",
    header=True,
    inferSchema=True)

df.show()
df.printSchema()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|          3.5|    151748|    98740|        8300|                High|
|      i8|2013|North America|   Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|7 Series|2020|South America| Black|   Diesel|      Manual|          2.1|    122131|    49898|   

In [ ]:
df_clean = df.dropna()
df_filled = df.fillna(0)

In [ ]:
from pyspark.sql.functions import col
# Example fix: Replace "column_name" with an actual column from your DataFrame, e.g., "Price_USD"
df = df.withColumn("Price_USD", col("Price_USD").cast("integer"))

In [ ]:
df = df.filter(df["Sales_Volume"] > 2000) # Example: filtering by 'Sales_Volume' being greater than 2000
df.show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|          3.5|    151748|    98740|        8300|                High|
|      i8|2013|North America|   Red|   Hybrid|   Automatic|          1.6|    121671|    79219|        3428|                 Low|
|5 Series|2022|North America|  Blue|   Petrol|   Automatic|          4.5|     10991|   113265|        6994|                 Low|
|      X3|2024|  Middle East|  Blue|   Petrol|   Automatic|          1.7|     27255|    60971|        4047|                 Low|
|7 Series|2020|South America| Black|   Diesel|      Manual|          2.1|    122131|    49898|   

In [ ]:
df.printSchema()

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Engine_Size_L: double (nullable = true)
 |-- Mileage_KM: integer (nullable = true)
 |-- Price_USD: integer (nullable = true)
 |-- Sales_Volume: integer (nullable = true)
 |-- Sales_Classification: string (nullable = true)



In [ ]:
print("Total records:", df.count())


Total records: 50000


In [ ]:
df.describe().show()

+-------+--------+-----------------+-------------+-----+---------+------------+------------------+------------------+------------------+------------------+--------------------+
|summary|   Model|             Year|       Region|Color|Fuel_Type|Transmission|     Engine_Size_L|        Mileage_KM|         Price_USD|      Sales_Volume|Sales_Classification|
+-------+--------+-----------------+-------------+-----+---------+------------+------------------+------------------+------------------+------------------+--------------------+
|  count|   50000|            50000|        50000|50000|    50000|       50000|             50000|             50000|             50000|             50000|               50000|
|   mean|    NULL|        2017.0157|         NULL| NULL|     NULL|        NULL| 3.247179999999999|      100307.20314|        75034.6009|        5067.51468|                NULL|
| stddev|    NULL|4.324459218093149|         NULL| NULL|     NULL|        NULL|1.0090783975411621|57941.50934352461

In [ ]:
df.filter(df.Region == "South America").orderBy(F.desc("Sales_Volume")).show()

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2024|South America| White| Electric|      Manual|          2.5|     52062|    88572|        9999|                High|
|      M5|2019|South America|  Grey|   Petrol|      Manual|          3.8|    141592|   105893|        9998|                High|
|      M3|2010|South America|  Blue|   Petrol|   Automatic|          4.3|    191728|    54325|        9996|                High|
|      i3|2018|South America|   Red|   Diesel|      Manual|          1.8|    188763|    46568|        9996|                High|
|      i8|2013|South America| Black|   Hybrid|      Manual|          2.1|     96520|    48851|   

In [ ]:
df.createOrReplaceTempView("bmw_data")

In [ ]:
sales = spark.read.csv("/content/bmw_data.csv")

sales.printSchema()
sales.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|     _c0| _c1|          _c2|   _c3|      _c4|         _c5|          _c6|       _c7|      _c8|         _c9|                _c10|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|          3.5|    151748|    9874

In [ ]:
sales.createOrReplaceGlobalTempView("car_sales")
df.write.mode("overwrite").saveAsTable("car_sales") #Save as table and overwrite table if exits

In [ ]:
df2 = spark.sql("SELECT * FROM car_sales")
df2.printSchema()
df2.show()

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Engine_Size_L: double (nullable = true)
 |-- Mileage_KM: integer (nullable = true)
 |-- Price_USD: integer (nullable = true)
 |-- Sales_Volume: integer (nullable = true)
 |-- Sales_Classification: string (nullable = true)

+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|   Model|Year|       Region| Color|Fuel_Type|Transmission|Engine_Size_L|Mileage_KM|Price_USD|Sales_Volume|Sales_Classification|
+--------+----+-------------+------+---------+------------+-------------+----------+---------+------------+--------------------+
|5 Series|2016|         Asia|   Red|   Petrol|      Manual|          3.5|    151748|    98740|        8300|                High|
|      i8|2013|No

In [ ]:
df3 = spark.sql("SELECT Year, SUM(Price_USD) AS total_023_sales FROM car_sales GROUP BY year ORDER BY SUM(Price_USD) DESC LIMIT 5")
df3.printSchema()
df3.show()

root
 |-- Year: integer (nullable = true)
 |-- total_023_sales: long (nullable = true)

+----+---------------+
|Year|total_023_sales|
+----+---------------+
|2022|      213012562|
|2024|      209595431|
|2019|      206642969|
|2016|      206326338|
|2021|      205447353|
+----+---------------+

